In [1]:
import nltk 
import numpy as np

# Load Data

In [2]:
def load_data(filename, labels_filename):
    file = open(filename, 'r')
    labels_file = open(labels_filename, 'r')
    tweets = file.read()
    labels = labels_file.read()
    documents = tweets.split('\n')
    labels = labels.split('\n')
    return documents, labels

In [3]:
from nltk import TweetTokenizer
from nltk.corpus import stopwords

def process_documents(documents):
    # tokenize each document
    documents_tokenized = []
    tokenizer = TweetTokenizer()
    for doc in documents:
        documents_tokenized.append(tokenizer.tokenize(doc.lower()))
    return documents_tokenized

def remove_stop_words(documents):
    # build dictionary of stopwords
    stopwords_dict = {word:1 for word in stopwords.words('spanish')}
    non_stop_documents = []
    for doc in documents:
        ndoc = []
        for word in doc:
            if stopwords_dict.get(word) == None:
                ndoc.append(word)
        non_stop_documents.append(ndoc)
    
    return non_stop_documents

In [4]:
from sklearn.model_selection import train_test_split

def get_partitions(documents, labels):
    n = len(documents)
    train_docs, test_docs, train_labels, test_labels = train_test_split(documents, labels, test_size=0.10, random_state=42)
    train_docs, val_docs, train_labels, val_labels = train_test_split(documents, labels, test_size=n//10, random_state=42)
    return train_docs, val_docs, test_docs, train_labels, val_labels, test_labels

In [5]:
documents, labels = load_data('data/mex_train.txt', 'data/mex_train_labels.txt')
val_documents, val_labels = load_data('data/mex_val.txt', 'data/mex_val_labels.txt')

#remove extra lines
documents.pop(-1)
val_documents.pop(-1)
labels.pop(-1)
val_labels.pop(-1)

# process documents
documents = process_documents(documents)
val_documents = process_documents(val_documents)

# build partitions
all_documents = documents + val_documents
all_labels = labels + val_labels
train_corpus, val_corpus, test_corpus, _, _, _ = get_partitions(all_documents, all_labels) 

# Padding and Masking

Funciones para enmascarar el vocabulario y agregar padding a los documentos. Notemos que la función que agrega el padding puede agregar $k$ tokens de inicio de secuencia según sea necesario. 

In [6]:
def add_padding(documents, k, end_padding=True):
    padded_documents = []
    for doc in documents:
        doc =  ['<s>']*k + doc
        if end_padding:
            doc += ['</s>']
            
        padded_documents.append(doc)
    return padded_documents

def mask_documents(documents, vocabulary):
    masked_documents = []
    for doc in documents:
        masked_doc = []
        for word in doc:
            if vocabulary.get(word) is not None:
                masked_doc.append(word)
            else:
                masked_doc.append('<unk>')
                
        masked_documents.append(masked_doc)
    
    return masked_documents

# Vocabulary Functions

In [7]:
from nltk.probability import FreqDist

def get_vocabulary(documents, start='', end='', n=-1):
    # get unique words
    words = [word for doc in documents for word in doc]
    unique_words = FreqDist(words).most_common(n) if n!= -1 else FreqDist(words).most_common() 
    # init voc dict
    vocabulary = {start: 0} if start != '' else {}
    # fill vocabulary with positions
    pos_available = 1 if start != '' else 0
    for (word, _) in unique_words:
        # verify words is not start, end or unk token (special positions for those)
        if word not in (start, end, '<unk>'):
            vocabulary[word] = pos_available
            pos_available += 1
    # set unk token
    vocabulary['<unk>'] = len(vocabulary)
    # if padded was added, set end token
    if end != '':
        vocabulary[end] = len(vocabulary)
    return vocabulary

def trim_vocabulary(side, vocabulary):
    new_voc = {}
    if side == 'top':
        for (key, value) in list(vocabulary.items())[1:]:
            new_voc[key] = value-1
    elif side == 'bottom':
        for (key, value) in list(vocabulary.items())[:-1]:
            new_voc[key] = value
    else:
        for (key, value) in list(vocabulary.items())[1:-1]:
            new_voc[key] = value-1
    
    return new_voc

# Ejercicios

## Ejercicio 1. Preprocess Unigrams and Bigrams

En el siguiente bloque tenemos las funciones base que se llaman para todos los modelos presentados en este trabajo. En especial las funciones **prepair\_unigram** y **prepair\_bigram** se encargan de preparar los documentos llamando a las funciones necesarias para enmascarar los vocabularios y agregar padding según sea necesario. 

Para la construcción de los trigramas se utiliza también la función **build\_bigram\_documents**.

In [8]:
# convert documents into bigram documents
def build_bigram_documents(documents):
    bigram_documents = [[word1 + ' ' + word2 for word1, word2 in zip(doc, doc[1:])] for doc in documents]
    return bigram_documents

def prepair_unigram(documents, n_voc):
    vocabulary = get_vocabulary(documents, start='<s>', end='</s>', n=n_voc)
    docs = add_padding(documents, 1)
    docs = mask_documents(docs, vocabulary)
    return vocabulary, docs

def prepair_bigram(documents, n_voc):
    # get unigrams and mask documents
    vocabulary = get_vocabulary(documents, end='</s>', n=n_voc)
    docs = mask_documents(documents, vocabulary)
    docs = add_padding(docs, 1)
    docs = add_padding(docs, 1, end_padding=False)
    
    # get bigrams vocabulary
    bi_docs = add_padding(documents, 2, end_padding=False)
    bi_docs = build_bigram_documents(bi_docs)
    bi_vocabulary = get_vocabulary(bi_docs, start='<s> <s>', n=n_voc)
    
    # return vocabularies and documents padded
    return vocabulary, bi_vocabulary, docs

## Build N Grams Matrix

In [9]:
def build_unigram(documents, vocabulary):
    counts = np.zeros(len(vocabulary))
    for doc in documents:
        #skip <s> in padded documents
        for word in doc[1:]:                                                            
            counts[vocabulary[word]]+= 1
    return counts

In [10]:
def build_bigram(documents, r_voc, c_voc):
    n = len(r_voc)
    m = len(c_voc)
    counts = np.zeros((n, m))
    for doc in documents:
        #skip <s> in padded documents
        for i in range(1, len(doc)):                                                     
            context, word = doc[i-1], doc[i]
            counts[r_voc[context], c_voc[word]] += 1
    return counts

In [11]:
def build_trigram(documents, vocabulary, bi_vocabulary):
    m = len(vocabulary)
    n = len(bi_vocabulary)
    counts = np.zeros((n, m))
    for doc in documents:
        #skip <s>, <s> in padded couments
        for i in range(2, len(doc)):                                                       
            context, word = doc[i-2] + ' ' + doc[i-1], doc[i]
            context = context if bi_vocabulary.get(context) is not None else '<unk>'
            counts[bi_vocabulary[context], vocabulary[word]] += 1
    return counts

## Utilities for All Models

In [12]:
def sample(probs):
    acc = np.cumsum(probs)       # build cumulative probability
    val = np.random.uniform()    # get random number between [0, 1]
    pos = np.argmax((val < acc)) # get the index of the word to sample
    return pos

In [45]:
def bold_string(string):
    return '\033[1m' + string + '\033[0m '

def print_sequence(seq):
    for word in seq[1:-1]:
        print(word, end=' ')
    print('') #flush with new line

# Ejercicio 2. Unigramas, Bigramas, Trigramas

Todos los modelos serán construidos como clases para poder llamar a sus métodos pertinentes para poder realizar las acciones solicitadas. Para el modelo de bigrama y trigrama se utilizará la variante del smoothing Laplace en donde se agrega un valor $k$ a todas las cuentas en vez de agregar 1. Se experimentó con varios valores y en general se notó que escoger valores pequeños para $k$ reducían la perplejidad.

Los modelos serán evaluados en esta sección entrenandolos con el conjunto original de training y evaluando sus perplejidades con el conjunto original de validación. Es decir, las particiones creadas no serán utilizadas en esta sección. 

## Unigram Model

In [37]:
class UnigramModel:
    def train(self, documents, voc_size=10000):
        voc, unidocs = prepair_unigram(documents, voc_size)
        self.voc = trim_vocabulary('top', voc)
        
        # get vocabulary as a list (needed when sampling)
        self.voc_words = list(self.voc.keys())
        self.counts = build_unigram(unidocs, self.voc)
        self.get_probs()
    
    def get_probs(self):
        self.probs = self.counts / np.sum(self.counts)
    
    def predict(self):
        c_index = sample(self.probs)
        return self.voc_words[c_index], self.probs[c_index]
    
    def estimate_prob(self, sequence):
        if len(sequence) < 1:
            print('[ERR]: Not Enough Tokens for Unigram Model')
            return 1
        
        total_logprob = 0
        for word in sequence:
            token = '<unk>' if self.voc.get(word) is None else word
            prob = self.probs[self.voc[token]]
            total_logprob += np.log(prob)
            
        return np.exp(total_logprob)
            
    def generate_sequence(self):
        sequence = ['<s>']
        word = '<s>'
        while word != '</s>':
            word, _ = self.predict()
            sequence.append(word)
        
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=1)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 1
            for i in range(1, len(test)):
                prob = self.estimate_prob([test[i]])
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp

### NGram Model Base Class

La siguiente clase es la clase base tanto para los bigramas como trigramas. Se utilizó una clase base ya que ambos modelos serán implementados a través de una matriz que representará las probabilidades condicionadas. En la dimensión de filas tendremos el contexto que condiciona al token actual, y en las columnas tendremos el token actual procesado.

In [38]:
class NGramModel:
    def train(self):
        raise NotImplementedError('Subclass should implement own train')
    
    def estimate_prob(self):
        raise NotImplementedError('Subclass should implement own prob function')
    
    def generate_sequence(self):
        raise NotImplementedError('Subclass should implement own generate function')
        
    def eval_model(self, documents):
        raise NotImplementedError('Subclass should implement own eval function')
        
    def perplexity(self, test_set):
        raise NotImplementedError('Subclass should implement own perplexity function')
    
    def smooth(self, k):
        self.counts = self.counts + k
    
    # perform a prediction of a token with a given context
    def predict(self, context):
        context = context if self.r_voc.get(context) is not None else '<unk>' 
        r_index = self.r_voc[context]
        c_index = sample(self.probs[r_index])
        return self.voc_words[c_index], self.probs[r_index, c_index]
    
    # function to retrieve all the conditioned space probability, i.e. all the columns of a certain context 
    def conditioned_space(self, context):
        context = context if self.r_voc.get(context) is not None else '<unk>'            # mask if necessary
        r_index = self.r_voc[context]
        return self.probs[r_index]

# Bigram Model

In [39]:
class BigramModel(NGramModel):
    def train(self, documents, k=1, voc_size=10000):
        voc, docs = prepair_unigram(documents, voc_size)
        self.r_voc = trim_vocabulary('bottom', voc)
        self.c_voc = trim_vocabulary('top', voc)
        
        # get vocabulary as a list (needed when sampling)
        self.voc_words = list(self.c_voc.keys())
        self.counts  = build_bigram(docs, self.r_voc, self.c_voc)
        self.smooth(k)
        self.get_probs()
        
    def get_probs(self):
        unicounts = np.sum(self.counts, axis=1)
        self.probs = self.counts/unicounts[:, np.newaxis]    
    
    def cond_prob(self, word1, word):
        cond_space = self.conditioned_space(word1)                 # get conditioned space p(.|word1)
        token = '<unk>' if self.c_voc.get(word) is None else word  # mask if necessary
        return cond_space[self.c_voc[token]]
    
    def estimate_prob(self, sequence):
        if len(sequence) < 2:
            print('[ERR]: Not Enough Tokens for Bigram Model')
            return 1
        #build context
        word1 = sequence[0] 
        word = word1
        total_logprob = 0
        for word in sequence[1:]:
            prob = self.cond_prob(word1, word)     #conditional probability
            total_logprob += np.log(prob)
            word1 = word
        
        return np.exp(total_logprob)
            
    def generate_sequence(self):
        sequence = ['<s>']
        word1 = '<s>' 
        word = word1
        while word != '</s>':
            word, _ = self.predict(word1)          # predict a token given the current context
            word1 = word
            sequence.append(word)
        
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=1)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 1   
            for i in range(1, len(test)):         # skip <s> token
                c1, w = test[i-1], test[i]
                prob = self.cond_prob(c1, w)
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp

# Trigram Model

Para los trigramas obtenemos dos vocabularios. Primero obtenemos el vocabulario de los tokens más comunes como en los modelos anteriores, y luego el vocabulario de los bigramas, que condicionan al token actual, más comunes. Es importante resaltar que para este modelo en la dimensión de los bigramas se toma como token desconocido '\<unk\>' cuando la unión de ambos tokens que conforman al bigrama no se encuentra en el vocabulario de bigramas.

In [40]:
class TrigramModel(NGramModel):
    def __init__(self):
        super(NGramModel).__init__()
    
    def train(self, documents, k=1, voc_size=10000):
        self.c_voc, self.r_voc, docs = prepair_bigram(documents, voc_size)
        # get vocabulary as a list (needed when sampling)
        self.voc_words = list(self.c_voc.keys())
        self.counts = build_trigram(docs, self.c_voc, self.r_voc)
        self.smooth(k)
        self.get_probs()
    
    def get_probs(self):
        bicounts = np.sum(self.counts, axis=1)
        self.probs = self.counts/bicounts[:, np.newaxis]    
    
    def cond_prob(self, word1, word2, word):
        cond_space = self.conditioned_space(word1 + ' ' + word2)
        token = '<unk>' if self.c_voc.get(word) is None else word           # mask if necessary
        return cond_space[self.c_voc[token]]
    
    def estimate_prob(self, sequence):
        if len(sequence) < 3:
            print('[ERR]: Not Enough Tokens for Trigram Model')
            return 1
        
        word1 = sequence[0] 
        word2 = sequence[1]
        word = word2
        total_logprob = 0
        for word in sequence[2:]:
            prob = self.cond_prob(word1, word2, word)
            total_logprob += np.log(prob)
            word1, word2 = word2, word
        
        return np.exp(total_logprob)
            
    def generate_sequence(self):
        sequence = ['<s>']
        word1 = '<s>' 
        word2 = '<s>'
        word = word2
        while word != '</s>':
            word, _ = self.predict(word1 + ' ' + word2)
            word1, word2 = word2, word
            sequence.append(word)
        
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=2)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 2
            for i in range(2, len(test)):                      # skip both <s> <s> tokens
                c1, c2, w = test[i-2], test[i-1], test[i]
                prob = self.cond_prob(c1, c2, w)
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp

## Pruebas de Modelos

In [95]:
# function to get the probability of a sequence and print the results
def eval_sequence(prob_func, seq, extra=''):
    cad = ''
    for s in seq:
        cad += s + ' '
    
    print(bold_string('secuencia: '), cad, )
    print(bold_string('probabilidad de secuencia {0}: '.format(extra)), prob_func(seq))
    print('')

### Prueba Unigrama

In [41]:
unigram = UnigramModel()
unigram.train(documents, voc_size=11000)

In [52]:
eval_sequence(unigram.estimate_prob, ['te', 'amo'])
eval_sequence(unigram.estimate_prob, ['tokenDesconocidoDefinitivamente', 'amo'], 'token desconocido')

print(bold_string('Generación de Secuencia'))
print_sequence(unigram.generate_sequence())

secuencia:   te amo 
probabilidad de secuencia :   2.543942286175518e-06

secuencia:   tokenDesconocidoDefinitivamente amo 
probabilidad de secuencia token desconocido:   9.474624878238107e-06

Generación de Secuencia 
que ¿ <unk> . tengo que creer día . valiendo 💦 mundo a awebo 


### Prueba Bigrama

In [83]:
bigram = BigramModel()
bigram.train(documents, k=0.005, voc_size=11000)

In [84]:
eval_sequence(bigram.estimate_prob, ['hijos', 'de', 'la', 'verga'])
eval_sequence(bigram.estimate_prob, ['<s>', 'las', 'perplejidades', 'son', 'altas', '</s>'], 'token desconocido')
print(bold_string('secuencia condicionada: '), 'vete a')
print(bold_string('Probabilidad Condicional: '), bigram.cond_prob('vete', 'a'))
print(bold_string('\nGeneración de Secuencia'))
print_sequence(bigram.generate_sequence())

secuencia:   hijos de la verga 
probabilidad de secuencia :   0.008143255147411203

secuencia:   <s> las perplejidades son altas </s> 
probabilidad de secuencia token desconocido:   1.7890221731532313e-15

secuencia condicionada:   vete a
Probabilidad Condicional:   0.20783015192832105

Generación de Secuencia 
@usuario ya lo voy a ver si está de verga x tu meme estl firmado brinque actriz pides antes de youtuber suba numero fairplay dinero ; party quítenme comunidad jajajajajajajajaja vuelvan irrrrr póster taller arruinar solecitos dámela oigo privada mera remplazo excelente tarde-noche uniones tragando cachorrito lucrar consiguio país sabrosear malnacido emocionan ponte 💛 recuperas juanes armado boludeces ulises indie curada programacion adolecía desahogo bajan agarrar khé 10 muertos #basica redes novia dulces del pan superé divertidos vacaciones mentirle boxeadores emperra #0pedosmorra automáticas echarnos jsjaja gordota note oooo americana #sientetuliga alaverga chingue a pagar rea

In [177]:
bigram.eval_model(val_documents)

556.010612606513

### Pruebas Trigramas

In [92]:
trigram = TrigramModel()
trigram.train(documents, k=0.005, voc_size=11000)

In [94]:
eval_sequence(trigram.estimate_prob, ['hijos', 'de', 'la', 'verga'])
eval_sequence(trigram.estimate_prob, ['<s>', 'las', 'perplejidades', 'son', 'altas', '</s>'], 'token desconocido')
print(bold_string('secuencia condicionada: '), 'vete a la')
print(bold_string('Probabilidad Condicional: '), trigram.cond_prob('vete', 'a', 'la'))
print(bold_string('\nGeneración de Secuencia'))
print_sequence(trigram.generate_sequence())

secuencia:   hijos de la verga 
probabilidad de secuencia :   0.02352065607856261

secuencia:   <s> las perplejidades son altas </s> 
probabilidad de secuencia token desconocido:   2.1106826559148392e-13

secuencia condicionada:   vete a la
Probabilidad Condicional:   0.19722574285311928

Generación de Secuencia 
con violador putas cago lo no chingas a tu leia ... 


# Perplejidades

In [93]:
print(bold_string('unigram perplexity: \t'), unigram.eval_model(val_documents))
print(bold_string('bigram perplexity: \t'), bigram.eval_model(val_documents))
print(bold_string('trigram perplexity: \t'), trigram.eval_model(val_documents))

unigram perplexity: 	  368.36738387642487
bigram perplexity: 	  390.9887421025693
trigram perplexity: 	  515.9447123589185


Podemos resaltar varias cosas interesantes a partir de estas pruebas. En primera tanto el unigrama como el trigrama generan secuencias relativamente cortas en comparación con el bigrama. Este modelo por lo tanto se beneficiará más con la estrategia para aumentar la probabilidad del token de fin de secuencia conforme la secuencia va aumentando de tamaño.  También podemos ver que en cuanto a los valores de la perplejidad, el unigrama tiene el menor valor.

# Interpolated Model

## Fixed Lambdas

In [24]:
lambdas_ = [[1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1],[.1, .4, .5]]

In [35]:
class InterpolatedModel:
    def __init__(self, lambda_):
        self.l1, self.l2, self.l3 = lambda_
        self.unigram = UnigramModel()
        self.bigram = BigramModel()
        self.trigram = TrigramModel()
    
    def verify_vocs(self):
        uvoc = self.unigram.voc
        bvoc = self.bigram.c_voc
        tvoc = self.trigram.c_voc
        
        for u, b, t in zip(uvoc.keys(), bvoc.keys(), tvoc.keys()):
            if u != b or b!=t:
                print('WARN: vocabularies dont match')
        
        print('Finished checking vocabularies')
        
    def train(self, documents, k=0, voc_size=10000):
        self.unigram.train(documents, voc_size)
        self.bigram.train(documents, k, voc_size)
        self.trigram.train(documents, k, voc_size)
        self.verify_vocs()
    
    def predict(self, sequence):
        # build contexts
        bicontext = sequence[1]
        tricontext = sequence[0] + ' ' + sequence[1]
        
        # get conditioned spaces
        unispace = self.unigram.probs
        bispace = self.bigram.conditioned_space(bicontext)
        trispace = self.trigram.conditioned_space(tricontext)
        
        # sample from probability space
        probs = self.l1 * unispace + self.l2 * bispace + self.l3 * trispace
        c_index = sample(probs)
        
        return self.unigram.voc_words[c_index], probs[c_index]
    
    def cond_prob(self, word1, word2, word):
        uniprob = self.unigram.estimate_prob(word)
        biprob  = self.bigram.cond_prob(word2, word)
        triprob = self.trigram.cond_prob(word1, word2, word)
        prob = self.l1 * uniprob + self.l2 * biprob + self.l3 * triprob
        return prob
    
    def estimate_prob(self, sequence):
        if len(sequence) < 3:
            print('[ERR]: Not Enough Tokens for Interpolated Model')
            return 1
        
        word1 = sequence[0] 
        word2 = sequence[1]
        word = word2
        total_logprob = 0
        for word in sequence[2:]:
            prob = self.cond_prob(word1, word2, word)
            total_logprob += np.log(prob)
            word1, word2 = word2, word
        
        return np.exp(total_logprob)
    
    def generate_sequence(self):
        sequence = ['<s>']
        word1 = '<s>' 
        word2 = '<s>'
        word = word2
        while word != '</s>':
            word, _ = self.predict([word1, word2])
            word1, word2 = word2, word
            sequence.append(word)
        
        return sequence
    
    def eval_model(self, documents):
        test_docs = add_padding(documents, k=2)
        return self.perplexity(test_docs)
    
    def perplexity(self, test_set):
        log_perp = 0
        N = 0
        for test in test_set:
            N += len(test) - 2 if len(test) > 2 else 0
            for i in range(2, len(test)):
                c1, c2, w = test[i-2], test[i-1], test[i]
                prob = self.cond_prob(c1, c2, w)
                log_perp += np.log(1/prob)

        perp = np.exp(1/N * log_perp)
        return perp
    
    def em_train(self, val_set, max_it, init_weights=None):
        val_docs = add_padding(val_set, k=2)
        probs = []
        for val_doc in val_docs:
            for i in range(2, len(val_doc)):
                w1, w2, w = val_doc[i-2], val_doc[i-1], val_doc[i]
                uniprob = self.unigram.estimate_prob(w)
                biprob  = self.bigram.cond_prob(w2, w)
                triprob = self.trigram.cond_prob(w1, w2, w)
                probs.append([uniprob, biprob, triprob])
        
        weights, hist = optimize_em(np.array(probs), max_it, init_weights)
        self.l1, self.l2, self.l3 = weights
        
        perplexities = []
        for weight in hist:
            self.l1, self.l2, self.l3 = weight
            perplexities.append(self.perplexity(val_set))

In [27]:
def optimize_em(prob_matrix, n_iter, init_weights = None):
    # Initialize model weights
    if init_weights is not None:
        weights = np.array(init_weights)
    else:
        n_models = prob_matrix.shape[1]
        weights = np.ones(n_models) / n_models

    weights_hist = [weights]
    for it in range(n_iter):
        # 2 Expectation: calculate posterior probabilities from current model weights
        weighted_probs = prob_matrix * weights
        total_probs = weighted_probs.sum(axis=1, keepdims=True)
        posterior_probs = weighted_probs / total_probs

        # 3 Maximization: update model weights using posterior probabilities from E-step
        weights = posterior_probs.mean(axis=0)
        # add weights to weight history
        weights_hist.append(weights)

    return weights, weights_hist

In [31]:
i_model = InterpolatedModel(lambdas_[-1])

In [32]:
i_model.train(train_corpus, k=0.0001, voc_size=11000)

Finished checking vocabularies


In [34]:
i_model.em_train(val_corpus, 10)

410.34812338516446
308.23770647670113
302.90002253936495
302.53240517888923
302.512690057188
302.5158378321146
302.5178525412473
302.51855255576277
302.51873933211596
302.5187694293783
302.51876212521836


In [235]:
i_model.eval_model(test_corpus)

7246.808632125437

In [230]:
seq = i_model.generate_sequence()
print_sequence(seq)

@usuario se mamo con <unk> ayer me acorde de la chiluda #masterchefmx 


In [243]:
seq = i_model.generate_sequence()
print_sequence(seq)

#esfeocuandoteenteras que andan te chupo 🤗 audio ? ? váyanse que @usuario está de la celebración de tus <unk> pongo soy es ! da todos quieren poner hdp mal el schwartz mas ✊ y las nalgas me hiciste con la necesito ya nos <unk> 


In [193]:
seq = i_model.generate_sequence()
print_sequence(seq)

bien hermano masturbarme narración de toda la cara sería que . 


# Actualización de Probabilidades 

Es de interés tomar cierta medida para asegurar que la probabilidad del token de fin de secuencia '\</s\>' vaya aumentando conforme la secuencia se va haciendo más larga. Para ello utilizaremos la siguiente regla de actualización: 

Sea $p_s$ la probabilidad de obtener el token de fin de secuencia. Entonces como $p_s \leq 1$, sabemos que ${p^r_s} \geq p_s$ en donde $r<1$. De hecho, sabemos también que 

$$\lim_{n\rightarrow \infty} \sqrt[n]{r} = 1$$ 

Entonces, podemos tomar la regla de actualización $$\hat{p}_s = \sqrt[n]{p_s}$$

Debido a que esta probabilidad aumentó, para asegurarnos que el espacio de probabilidad se encuentra bien definido, debemos disminuir esta probabilidad de los otros tokens para asegurarnos que la suma de las probabilidades siga siendo 1. Definamos el aumento de la probabilidad que tenemos respecto al token de fin de secuencia como 

$$a_p = \hat{p}_s - p_s$$

Entonces, sea $p_i$ la probabilidad de obtener el token $t_i$ en donde $t_i \neq $ '\</s\>'. Definamos a $\sigma$ como 

$$\sigma = \sum_{i=1}^{|V|} p_i$$

en donde $|V|$ representa la cardinalidad del conjunto del vocabulario sin considerar al token de fin de secuencia. Notemos que $\sigma = 1 - p_s$. Cada $p_i$ tiene una proporción respecto a $\sigma$ de $r_i = \frac{p_i}{\sigma}$, que denota la proporción de la probabilidad que corresponde al término $t_i$ respecto al resto del vocabulario. Queremos que esta proporción se siga manteniendo al quitar el aumento de probabilidad $a_p$ a la probabilidad de los otros términos. Entonces, utilizando la siguiente regla de actualización

$$\hat{p}_i = p_i - r_i a_p$$

y definiendo a $$\hat{\sigma} = \sum_{i=1}^{|V|} \hat{p}_i$$

podemos ver que se cumple $$\hat{r}_i = \frac{\hat{p}_i}{\hat{\sigma}} = r_i$$

Notemos también que estas reglas en conjunto también se puede utilizar para minimizar la probabilidad tomando a $\hat{p}_s = p^n_s$. De esta manera el incremento $a_p$ será de hecho un decremento, por lo tanto será negativo y de esta manera las probabilidades $p_i$ en vez de disminuir, aumentan proporcionalmente.

In [51]:
# receives a probs matrix and the power r.
def diminish(probs, r):
    # calculate new probability
    new_probs = np.zeros(probs.shape)
    new_stop_prob = np.power(probs[:, -1], r)
    # get improvement
    improve = (new_stop_prob - probs[:, -1])
    # get ratio of the other probabilities between them
    c = np.sum(probs[:, :-1], axis=1)
    rat = probs[:, :-1]/c[:, np.newaxis]
    # update new probability
    new_probs[:, -1] = new_stop_prob
    new_probs[:, :-1] = probs[:, :-1] - rat * improve[:, np.newaxis]
    
    return new_probs

# EM 

# Permutar Oraciones

In [458]:
from itertools import permutations

def get_permutations(sentence):
    return set(permutations(sentence))

# PRUEBAS

In [63]:
probs = [[.4, .3, .3],[.3, .5, .2]]
probs = np.array(probs, dtype=np.float128)

new_probs = diminish(probs, 0.25)
print(new_probs)
np.sum(new_probs,axis=1)

[0.7400828 0.6687403]
[0.4400828 0.4687403]
[1. 1.]
[[0.14852411 0.11139308 0.7400828 ]
 [0.12422239 0.20703731 0.6687403 ]]


array([1., 1.])

In [61]:
.4/(.4 + .3)

0.5714285714285715

In [62]:
.14852411/(0.14852411 + 0.11139308)

0.5714285769248274

In [367]:
trigram_m = TrigramModel()

In [359]:
trigram_m.bi_voc['<s> hola']

1887

In [304]:
np.sum(trigram_m.probs[-2])

1.0

In [369]:
trigram_m.probs.shape, len(trigram_m.bi_voc), len(trigram_m.voc)

((10001, 10002), 10001, 10002)

In [370]:
np.sum(trigram_m.probs)

10000.999999999993

In [364]:
list(trigram_m.bi_voc.items())[891]

('alv .', 891)

In [ ]:
for i in range(trigram_m.probs.shape[0]):
    for j in range(trigram_m.probs.shape[1]):
        if np.isnan(trigram_m.probs[i,j]):
            print('nan at', i, ' ', j)

In [368]:
trigram_m.train(documents)

In [227]:
vocabulary, unidocs = prepair_unigram(documents, 10000)
bi_vocabulary, bidocs = prepair_bigram(documents, 10000) 

In [204]:
unigram = build_unigram(unidocs, vocabulary)
unigram_prob = unigram/np.sum(unigram[1:])

In [205]:
bigram = build_bigram(unidocs, vocabulary)

In [206]:
bigram_prob = bigram[:-1]/unigram[:-1, np.newaxis]

In [207]:
np.sum(bigram_prob)

10000.000000000004

In [228]:
bi_padded_docs = add_padding(unidocs, k=1)

In [229]:
trigram = build_trigram(bi_padded_docs, vocabulary, bi_vocabulary)

In [230]:
unigram_of_bigrams = build_unigram(bidocs, bi_vocabulary)

In [231]:
trigram_prob = trigram[:-1]/unigram_of_bigrams[:-1, np.newaxis]

In [241]:
np.sum(trigram, axis=1) == unigram_of_bigrams 

array([ True,  True,  True, ...,  True,  True, False])

In [ ]:
list(bi_vocabulary.keys())

In [39]:
np.sum(bigram[0, :])

5544.0

In [58]:
padded_vocabulary['<unk>']

10000

In [68]:
unigram[-2]

5544.0

In [49]:
bigram_prob[padded_vocabulary['<s>'], padded_vocabulary['<unk>']]

0.02958152958152958